In [5]:
import pandas as pd
import joblib
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# =================================================================
# FASE 1: DESENVOLVIMENTO DO MODELO (Data Science)
# =================================================================

# #explorando os dados e fazendo a limpeza
# Lendo o dataset de treino (Buscape)
df = pd.read_csv('buscape.csv')

# Selecionando colunas relevantes e removendo valores nulos
df = df[['review_text', 'polarity']].dropna()

# Padronização básica: texto em minúsculo e sem espaços extras
df['review_text'] = df['review_text'].str.lower().str.strip()

# Definindo variáveis dependentes e independentes
X = df['review_text']
y = df['polarity']

# Divisão dos dados: 80% para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# #transformando textos em TF-IDF e #treinando o modelo supervisionado
# Criamos um Pipeline para garantir que o TF-IDF e a Regressão Logística sigam juntos
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression())
])

# Treinamento do modelo
print("Treinando o modelo... Aguarde.")
pipeline.fit(X_train, y_train)


# #criando métricas de desempenho
# Avaliando o quão bem o modelo aprendeu
y_pred = pipeline.predict(X_test)

print("\n--- Métricas de Desempenho (Base de Teste) ---")
print(f"Acurácia Geral: {accuracy_score(y_test, y_pred):.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


# #fazendo a serialização
# Salvando o pipeline completo para que o Back-end possa utilizar
model_filename = "modelo_sentimento_pipeline.joblib"
joblib.dump(pipeline, model_filename)
print(f"\nModelo serializado com sucesso: {model_filename}")


# =================================================================
# FASE 2: VALIDAÇÃO DA SERIALIZAÇÃO (Simulação de Uso Real)
# Nota: Este bloco simula como o modelo processará dados externos (JSON).
# =================================================================

print("\n" + "="*50)
print("TESTE DE INTEGRAÇÃO: PROCESSANDO ARQUIVO DATA.JSON")
print("="*50)

# 1. Carregando o arquivo serializado (Simulando o que o Back-end faria)
modelo_para_uso = joblib.load(model_filename)

# 2. Lendo o arquivo JSON enviado para o projeto
try:
    with open('data.json', 'r', encoding='utf-8') as f:
        dados_novos = json.load(f)

    # 3. Processando cada entrada do JSON com o modelo "congelado"
    for i, item in enumerate(dados_novos):
        texto_input = item['text']

        # Realizando a predição
        classe_predita = modelo_para_uso.predict([texto_input])[0]

        # Calculando a confiança (probabilidade)
        probabilidade = modelo_para_uso.predict_proba([texto_input]).max()

        # Traduzindo a classe numérica para algo legível (opcional)
        sentimento = "Positivo" if classe_predita == 1.0 else "Negativo"

        print(f"Item {i+1}:")
        print(f"Texto: {texto_input[:60]}...")
        print(f"Resultado do Modelo: {sentimento} (Confiança: {probabilidade:.2f})")
        print("-" * 30)

except FileNotFoundError:
    print("Erro: Arquivo 'data.json' não encontrado para o teste de validação.")

Treinando o modelo... Aguarde.

--- Métricas de Desempenho (Base de Teste) ---
Acurácia Geral: 0.93

Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.90      0.28      0.43       342
         1.0       0.93      1.00      0.96      3355

    accuracy                           0.93      3697
   macro avg       0.91      0.64      0.70      3697
weighted avg       0.93      0.93      0.91      3697


Modelo serializado com sucesso: modelo_sentimento_pipeline.joblib

TESTE DE INTEGRAÇÃO: PROCESSANDO ARQUIVO DATA.JSON
Item 1:
Texto: O atendimento foi excelente e a entrega chegou super rápida,...
Resultado do Modelo: Positivo (Confiança: 0.98)
------------------------------
Item 2:
Texto: O produto veio com defeito e a troca está demorando muito. P...
Resultado do Modelo: Negativo (Confiança: 0.84)
------------------------------
Item 3:
Texto: Chegou dentro do prazo, mas o manual de instruções é confuso...
Resultado do Modelo: Positivo 